# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)
print("Total number of files: ", len(file_path_list))

/home/workspace
Total number of files:  30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# print(full_data_rows_list[0])

In [4]:
print("Total number of rows: ",len(full_data_rows_list))

Total number of rows:  8056


In [5]:
# creating a smaller event data csv file called event_datafile_new csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    
    for row in full_data_rows_list:
        if (row[0] == ''):
            #skipping the row if artist column is empty
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print("Total number of rows after cleaning the data:",sum(1 for line in f))

Total number of rows after cleaning the data: 6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
# TO-DO: Create a Keyspace, just like database(name)  in RDBMS
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
## Creating Table for Query-1
query = "CREATE TABLE IF NOT EXISTS musicapp_history1"
query += '''( artist text, 
              item_in_session int, 
              length float, 
              session_id int, 
              song text,  
            PRIMARY KEY(session_id,item_in_session))'''
## PRIMARY KEY: combining session_id and item_in_session partion key will make the row unique for the above table and \
## we used the two partion keys in the where class in the mentioned above order.
try:
    session.execute(query)
except Exception as e:
    print(e)                  

In [11]:
def insert_data(file, session, query, indicies, dtypes):
    '''
    Creating pipeline to insert data into a Apache Cassandra table from CSV file.
    args:
        file(str): CSV file name
        session(object): Apache Cassandra session
        query(str): INSERT query statement 
        indicies(list): list of indicies to pass to INSERT statement
        dtypes(list): list of data types to convert indices values
    
    return: 
        None      
    '''
    #check both are of same size else stop executing the rest and throw an error
    assert(len(indicies) == len(dtypes))
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            session.execute(query, tuple(dtype(line[x]) for x, dtype in zip(indicies, dtypes)))  

In [12]:
# file name
file = 'event_datafile_new.csv'

# Query-1 indices and dtypes
indicies = [0, 3, 5, 8, 9]
dtypes = [str, int, float, int, str]

# INSERT query statement
query = "INSERT INTO musicapp_history1(artist, item_in_session, length, session_id, song)"
query = query + "VALUES(%s, %s, %s, %s,%s)"

# INSERT into the table
insert_data(file, session, query, indicies, dtypes)

In [13]:

# with open(file, encoding = 'utf8') as f:
#     csvreader = csv.reader(f)
#     next(csvreader) # skip header
#     for line in csvreader:
#         query = "INSERT INTO musicapp_history1(artist, item_in_session, length, session_id, song)"
#         query = query + "VALUES(%s, %s, %s, %s,%s)"
#         session.execute(query, (line[0], int(line[3]), float(line[5]), int(line[8]), line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
## This query will fetch the artist, song title and length of the song \
## of session id 338 and item session 4
query = """SELECT artist, song, length
           FROM musicapp_history1
           WHERE session_id = 338 AND item_in_session = 4"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# collecting all rows in list as a tuple
allRows = [] 
for row in rows:
    allRows.append((row.artist, row.song,row.length))

In [15]:
#creating dataFrame for the result query
df = pd.DataFrame(allRows,columns=['artist','song','length'])
# printing top 5 rows in the dataFrame
df.head()

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [16]:
## Creating Table for Query-2
query = "CREATE TABLE IF NOT EXISTS musicapp_history2"
query += '''( artist text,
              first_name text,
              item_in_session int,
              last_name text,
              session_id int, 
              song text,
              user_id int,
           PRIMARY KEY((user_id, session_id), item_in_session))'''
try:
    session.execute(query)
except Exception as e:
    print(e)            

In [17]:
# Query-2 indices and dtypes
indicies = [0, 1, 3, 4, 8, 9, 10]
dtypes = [str, str, int, str, int, str, int]

# INSERT query statement
query = "INSERT INTO musicapp_history2(artist, first_name, item_in_session, last_name, session_id, song, user_id)"
query = query + "VALUES(%s, %s, %s, %s,%s, %s,%s)"

# INSERT into the table
insert_data(file, session, query, indicies, dtypes)

In [18]:
## This query will fetch name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = """ SELECT artist, song, first_name, last_name
           FROM musicapp_history2
           WHERE user_id = 10 AND session_id = 182
           order by item_in_session DESC """

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# collecting all rows in list as a tuple
allRows = [] 
for row in rows:
    allRows.append((row.artist, row.song,row.first_name, row.last_name))

In [19]:
#creating dataFrame for the query result.
df = pd.DataFrame(allRows,columns=['artist','song','first_name','last_name'])
# printing top 5 rows in the dataFrame
df.head()

,artist,song,first_name,last_name
0,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz
1,Sebastien Tellier,Kilometer,Sylvie,Cruz
2,Three Drives,Greece 2000,Sylvie,Cruz
3,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz


In [20]:
## Creating Table for Query-3
query = "CREATE TABLE IF NOT EXISTS musicapp_history3"
query += '''( first_name text,
              item_in_session int,
              last_name text,
              session_id int, 
              song text,
              user_id int,
           PRIMARY KEY(song, session_id, item_in_session))'''
## PRIMARY KEY: combining song,  session_id and item_in_session partion keys will make the row unique for the above table and \
## used song in where clause to fetch the all the rows based on song title
try:
    session.execute(query)
except Exception as e:
    print(e)                

In [21]:
# Query-2 indices and dtypes
indicies = [1, 3, 4, 8, 9, 10]
dtypes = [str, int, str, int, str, int]

# INSERT query statement
query = "INSERT INTO musicapp_history3( first_name,item_in_session, last_name, session_id, song, user_id)"
query = query + "VALUES(%s,%s, %s, %s, %s,%s)"

# INSERT into the table
insert_data(file, session, query, indicies, dtypes)

In [22]:
## This query will fetch me user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = """ SELECT first_name, last_name
           FROM musicapp_history3
           WHERE song = 'All Hands Against His Own' """

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# collecting all rows in list as a tuple
allRows = []    
for row in rows:
    allRows.append((row.first_name, row.last_name))

In [23]:
#creating dataFrame for the result query
df = pd.DataFrame(allRows,columns=['first_name','last_name'])
# printing top 5 rows in the dataFrame
df.head()

,first_name,last_name
0,Sara,Johnson
1,Jacqueline,Lynch
2,Tegan,Levine


### Drop the tables before closing out the sessions

In [24]:
def drop_table(table_names):
    
    '''
    This function helps in drop the tables in the KEYSAPCE.
    arg:
        table_names(list): list of table names to drop in the KEYSPACE
    return:
        None
    '''
    for table in table_names:
        query = "DROP TABLE "+table
        try:
            session.execute(query)
        except Exception as e:
            print(e)

# List of table names to drop
table_names = ['musicapp_history1', 'musicapp_history2', 'musicapp_history3']

# Drop tables
drop_table(table_names)

### Close the session and cluster connection¶

In [25]:
session.shutdown()
cluster.shutdown()